In [6]:
import os, random, glob, sys
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

torch.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(linewidth=sys.maxsize)

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=sys.maxsize)

from core.model.Unet import UNet_2D_256
from core.utils.utils import import_mapf_instance, inverse_sigmoid, get_scene_80, get_sum_of_cost, get_cbs_result
from core.func.cbs_basic import CBSSolver, CBSSolver_Cost_Plot, CBSSolver_Cost
from core.func.icbs_complete import ICBS_Solver_Compare

In [7]:
def dense_input_data(starts, goals):
    input_data = []

    all_data = [(sx, sy, gx, gy) for (sx, sy), (gx, gy) in zip(starts, goals)]

    for i, (start, goal) in enumerate(zip(starts, goals)):
        sx, sy = start
        gx, gy = goal
        dx, dy = gx-sx, gy-sy
        mag = np.sqrt(dx ** 2 + dy ** 2)

        if mag != 0:
            dx = dx / mag
            dy = dy / mag

        except_agent_data = all_data[:i] + all_data[i+1:]

        own_data = [sx, sy, gx, gy, dx, dy, mag]

        for (other_sx, other_sy, other_gx, other_gy) in except_agent_data:
            other_dx, other_dy = other_gx-other_sx, other_gy-other_sy
            other_mag = np.sqrt(other_dx ** 2 + other_dy ** 2)

            if other_mag != 0:
                other_dx = other_dx / other_mag
                other_dy = other_dy / other_mag

            own_data += [other_sx-sx, other_sy-sy, other_dx, other_dy, other_mag]

        own_data = np.array(own_data)
        input_data.append(own_data)
    
    return np.array(input_data)

Sorted_Normalized_no_vertex

In [8]:
test_scene_list = get_scene_80("test_62", "bigger_heatmap", dataset="test")
method = "bigger_heatmap"
device = "cuda:0" if torch.cuda.is_available() else 'cpu'
print("device: {}".format(device))

Total 300 data loaded for test!
device: cuda:0


In [13]:
model = UNet_2D_256(2, 1, 202)
# model = cnn_mlp_64(102, 4096)
mseloss = nn.MSELoss()

model_name = "unet_256"
loss_name = mseloss._get_name()

# /home/railab/Workspace/CCBS/code/model_params/unet/MSELoss/L2_None/batch_32_lr_0.0001_epoch_1.pt
save_path = "/home/railab/Workspace/CCBS/code/model_params/{}/{}/L2_None".format(model_name, loss_name)

if os.path.exists(save_path):
    saved_model_list = glob.glob(save_path + "/batch*")
    saved_model_list.sort(key= lambda x : int(x.split("_")[-1][:-4]))
    last_model_file_name = saved_model_list[-1]
    previous_epochs = int(last_model_file_name.split("_")[-1][:-4])
else:
    previous_epochs = 0
    loss_list = []
    print("There is no trained model!")

print("Model: {}\nLoss: {}\n".format(model_name, loss_name))

Model: unet_256
Loss: MSELoss



In [14]:
model.load_state_dict(torch.load(last_model_file_name))

<All keys matched successfully>

In [15]:
avg_loss = 0.
model.eval()

for i, scene in enumerate(test_scene_list):
    # if "27104" not in scene:
    #     continue

    # Get data
    test_name = scene.split("/")[-2]
    scene_name = scene.split("/")[-1][:-4]

    total_result_file_name = "/home/railab/Workspace/CCBS/Train_results/Overfitting_test_62/cbs_results/bigger_heatmap/" + scene_name + ".txt"

    if os.path.exists(total_result_file_name):
        continue

    map, starts, goals = import_mapf_instance(scene)
    agent_num = len(starts)

    obs_map = np.array([[int(val) for val in sublist] for sublist in map])
    obs_map = np.repeat(obs_map[np.newaxis,:,:], agent_num, axis=0)

    heatmap_path = "/home/railab/Workspace/CCBS/code/costmaps/Gaussian_Blur/{}_heatmap/bigger_heatmap/{}.npy".format(test_name, scene_name)
    heatmap_data = np.load(heatmap_path, allow_pickle=True)

    h_map_path = "/home/railab/Workspace/CCBS/code/h_value_maps/{}/bigger_heatmap/{}.npy".format(test_name, scene_name)
    h_map_array = np.load(h_map_path, allow_pickle=True)

    input_data = dense_input_data(np.array(starts), np.array(goals))

    obs_map = torch.reshape(torch.Tensor(obs_map), heatmap_data.shape)
    h_val_map = torch.reshape(torch.Tensor(h_map_array), heatmap_data.shape)

    conv_data = torch.cat((obs_map, h_val_map), axis=1)

    input_data = torch.Tensor(input_data)

    pred = model(conv_data, input_data)
    pred = torch.reshape(pred, heatmap_data.shape)
    pred = torch.sigmoid(pred)
    # loss = mseloss(pred, torch.Tensor(heatmap_data))
    # print(loss)

    pred = pred.detach().numpy()

    pred = np.where(pred<1e-3, 0, pred)

    # print(pred)
    # print(heatmap_data)

    # print(pred.shape)
    # print(heatmap_data.shape)

    
    # /home/railab/Workspace/CCBS/code/instances/gen_test/test_69
    result_file_path = "/home/railab/Workspace/CCBS/code/instances/gen_test/{}/{}.txt".format(test_name, scene_name)
    print("==============================================================")
    print("Scene: {}".format(scene_name))

    f = open(result_file_path, 'r')
    contents = f.readlines()
    cbs_result = contents[1] + contents[2].rstrip()
    f.close()

    print("CBS " + contents[2].split(", ")[-1])
    
    # cbs_gen_nodes = int(contents[2].split(", ")[1].split(": ")[1])
    # cbs_exp_nodes = int(contents[2].split(", ")[2].split(": ")[1])

    # target_solver = CBSSolver_Cost(map, starts, goals, heatmap_data)
    # target_path, target_gen_nodes, target_exp_nodes, target_total_CT = target_solver.find_solution()
    # target_cost = get_sum_of_cost(target_path)

    # target_result = "\nTarget results!\nCost: {}, Gen_nodes: {}, Exp_nodes: {}, Total CT: {}".format(target_cost, target_gen_nodes, target_exp_nodes, target_total_CT)
    # print(target_result)

    # visualize_heatmap(heatmap_data, "target")
    # visualize_heatmap(pred, 'pred')

    l_cbs_solver = CBSSolver_Cost(map, starts, goals, pred)
    l_cbs_path, l_cbs_gen_nodes, l_cbs_exp_nodes, l_cbs_total_CT = l_cbs_solver.find_solution()

    if l_cbs_path != None:

        l_cbs_cost = get_sum_of_cost(l_cbs_path)

        l_cbs_result = "\nLearned_CBS results!\nCost: {}, Gen_nodes: {}, Exp_nodes: {}, Total CT: {}\n".format(l_cbs_cost, l_cbs_gen_nodes, l_cbs_exp_nodes, l_cbs_total_CT)
        total_result = "".join(contents) + l_cbs_result
        print(total_result)

        total_result_file_name = "/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/" + scene_name + ".txt"
        f = open(total_result_file_name, "w")
        f.write(total_result)
        f.close()
    else:
        continue
    

Scene: test_62_22955
CBS total CT: 5.7229225635528564

test_62_22955
CBS result!
Cost: 7514, Gen nodes: 27, Exp nodes: 14, total CT: 5.7229225635528564
ICBS result!
Cost: 7514, Gen nodes: 14, Exp nodes: 14, total CT: 27.410381078720093
Learned_CBS results!
Cost: 7514, Gen_nodes: 19, Exp_nodes: 10, Total CT: 3.393585443496704

Scene: test_62_22543
CBS total CT: 42.819053411483765

test_62_22543
CBS result!
Cost: 8152, Gen nodes: 91, Exp nodes: 46, total CT: 42.819053411483765
ICBS result!
Cost: 8152, Gen nodes: 48, Exp nodes: 46, total CT: 338.8443932533264
Learned_CBS results!
Cost: 8152, Gen_nodes: 37, Exp_nodes: 19, Total CT: 7.745630979537964

Scene: test_62_23001
CBS total CT: 88.3475136756897

test_62_23001
CBS result!
Cost: 8502, Gen nodes: 409, Exp nodes: 205, total CT: 88.3475136756897
ICBS result!
Cost: 8502, Gen nodes: 13, Exp nodes: 12, total CT: 24.94297981262207
Learned_CBS results!
Cost: 8502, Gen_nodes: 53, Exp_nodes: 27, Total CT: 11.717802286148071

Scene: test_62_2266

In [20]:
import glob


In [21]:
train_result_file_list = sorted(glob.glob("/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/*.txt"))
print(len(train_result_file_list))
train_result_file_list

298


['/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20789.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20792.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20793.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20801.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20806.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20809.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20811.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_20815.txt',
 '/home/railab/Workspace/CCBS/Results/Train_results/Overfitting_test_62/learned_cbs_results/test_62_2082

In [22]:
total_cbs_cost = 0
total_cbs_gen_nodes = 0
total_cbs_exp_nodes = 0
total_cbs_CT = 0

total_icbs_cost = 0
total_icbs_gen_nodes = 0
total_icbs_exp_nodes = 0
total_icbs_CT = 0

total_lcbs_cost = 0
total_lcbs_gen_nodes = 0
total_lcbs_exp_nodes = 0
total_lcbs_CT = 0

for train_result_file in train_result_file_list:
    f = open(train_result_file, 'r')
    result = f.readlines()
    f.close()

    # print(result)

    # print(train_result_file)

    cbs_result = result[2]
    icbs_result = result[4]
    lcbs_result = result[6]

    cbs_cost = int(cbs_result.split(", ")[0].split(": ")[-1])
    cbs_gen_nodes = int(cbs_result.split(", ")[1].split(": ")[-1])
    cbs_exp_nodes = int(cbs_result.split(", ")[2].split(": ")[-1])
    cbs_total_CT = float(cbs_result.split(", ")[3].split(": ")[-1])

    icbs_cost = int(icbs_result.split(", ")[0].split(": ")[-1])
    icbs_gen_nodes = int(icbs_result.split(", ")[1].split(": ")[-1])
    icbs_exp_nodes = int(icbs_result.split(", ")[2].split(": ")[-1])
    icbs_total_CT = float(icbs_result.split(", ")[3].split(": ")[-1])

    lcbs_cost = int(lcbs_result.split(", ")[0].split(": ")[-1])
    lcbs_gen_nodes = int(lcbs_result.split(", ")[1].split(": ")[-1])
    lcbs_exp_nodes = int(lcbs_result.split(", ")[2].split(": ")[-1])
    lcbs_total_CT = float(lcbs_result.split(", ")[3].split(": ")[-1])

    total_cbs_cost += cbs_cost
    total_cbs_gen_nodes += cbs_gen_nodes
    total_cbs_exp_nodes += cbs_exp_nodes
    total_cbs_CT += cbs_total_CT

    total_icbs_cost += icbs_cost
    total_icbs_gen_nodes += icbs_gen_nodes
    total_icbs_exp_nodes += icbs_exp_nodes
    total_icbs_CT += icbs_total_CT

    total_lcbs_cost += lcbs_cost
    total_lcbs_gen_nodes += lcbs_gen_nodes
    total_lcbs_exp_nodes += lcbs_exp_nodes
    total_lcbs_CT += lcbs_total_CT

avg_cbs_cost = total_cbs_cost / len(train_result_file_list)
avg_cbs_gen_nodes = total_cbs_gen_nodes / len(train_result_file_list)
avg_cbs_exp_nodes = total_cbs_exp_nodes / len(train_result_file_list)
avg_cbs_CT = total_cbs_CT / len(train_result_file_list)

avg_icbs_cost = total_icbs_cost / len(train_result_file_list)
avg_icbs_gen_nodes = total_icbs_gen_nodes / len(train_result_file_list)
avg_icbs_exp_nodes = total_icbs_exp_nodes / len(train_result_file_list)
avg_icbs_CT = total_icbs_CT / len(train_result_file_list)

avg_lcbs_cost = total_lcbs_cost / len(train_result_file_list)
avg_lcbs_gen_nodes = total_lcbs_gen_nodes / len(train_result_file_list)
avg_lcbs_exp_nodes = total_lcbs_exp_nodes / len(train_result_file_list)
avg_lcbs_CT = total_lcbs_CT / len(train_result_file_list)

print("CBS\nAvg cost: {}, Avg gen nodes: {}, Avg exp nodes: {}, Avg CT: {}".format(avg_cbs_cost, avg_cbs_gen_nodes, avg_cbs_exp_nodes, avg_cbs_CT))
print("\nICBS\nAvg cost: {}, Avg gen nodes: {}, Avg exp nodes: {}, Avg CT: {}".format(avg_icbs_cost, avg_icbs_gen_nodes, avg_icbs_exp_nodes, avg_icbs_CT))
print("\nLCBS\nAvg cost: {}, Avg gen nodes: {}, Avg exp nodes: {}, Avg CT: {}".format(avg_lcbs_cost, avg_lcbs_gen_nodes, avg_lcbs_exp_nodes, avg_lcbs_CT))
    

CBS
Avg cost: 7859.875838926175, Avg gen nodes: 149.58389261744966, Avg exp nodes: 75.29194630872483, Avg CT: 48.13262438054053

ICBS
Avg cost: 7859.879194630873, Avg gen nodes: 26.818791946308725, Avg exp nodes: 23.41275167785235, Avg CT: 159.50145258519473

LCBS
Avg cost: 7859.875838926175, Avg gen nodes: 57.630872483221474, Avg exp nodes: 29.315436241610737, Avg CT: 29.908579320715578
